组态选择需要从mix**2数据从高到低选择，先从90%开始，然后95%
然后在从中间态开始选择
连个选择先给出组态的索引列表，然后通过合并两个列表得到最终的组态索引列表。

运行流程：
1. 读取CSFs文件、MIX文件；
2. 使用batch_asfs_mix_square_above_threshold函数提取每个能级贡献靠前的mix的索引
3. 使用CSF_final_coupling_J_collection函数找出中间耦合中贡献较高的csfs的索引
4. 新建一个函数，用于合并两个索引，这个需要考虑计算中所包含的每个block中的所有能级
5. 通过rcsfzerofirst排列已有和为计算的csfs，提取其中的位置信息
6. 通过zerofirst的位置信息，设置函数随机或按照CSF_final_coupling_J_collection中提取的csfs占比来提取csfs
7. 正常计算流程
8. 提取新的mix结果与原mix结果对比（应该使用rlevel的结果，要保证最终的组态正确，尝试在dataio.py中写新的函数用于对比rlevel结果）
9. 循环6、7、8获得一个迭代收敛的结果




## 20250518
block_middle_coupling_theme -> all CSFs
用于选择初始计算的CSFs

todo list:
CSFs zero first sorted.
zero level CSFs sorted by mix coefficient from high to low.

现在需要随机提取csfs并追加在csfs文件末尾，并给出索引信息（csv？或者二进制）
读取rlevel导出的文件，后面使用rlevel导出时直接使用level这个后缀名，用于校验前后两次计算结果的谱项是否一致

In [1]:
import yaml
import argparse
import logging
from types import SimpleNamespace
import os
from pathlib import Path
import csv
import sys
import math
import numpy as np
import pandas as pd
import time
import joblib
import json 
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
sys.path.append('/Users/yiqin/Documents/PythonProjects/GraspDataProcessing/src')
import graspdataprocessing as gdp

In [2]:
config_file = 'config.yaml'  # 配置文件的路径
configasdfas = gdp.load_config(config_file)

配置验证通过: cutoff_value=1e-09, chosen_ratio=0.09


In [2]:
ag = '5s   4d-  4d   5p-  5p   6s   4f-  4f   5d-  5d   6p-  6p   7s   5f-  5f   6d-  6d   7p-  7p   8s   5g-  5g   6f-  6f   7d-  7d   8p-  8p   9s   6g-  6g   7f-  7f   8d-  8d   9p-  9p  10s'

In [3]:
peel_subshells = ag.split()
csf_data = [
    '  5s ( 2)  4d-( 4)  4d ( 6)  5p-( 2)  5p ( 4)  6s ( 2)  4f-( 3)  4f ( 4)  5d-( 1)',
    '                                                            9/2        8      3/2',
    '                                                                        7/2      2-'
]

In [4]:
asdg = gdp.parse_csf_2_descriptor_with_subshell(peel_subshells, csf_data)

In [6]:
asdg

array([ 5., -1.,  2.,  0.,  0.,  4.,  2.,  4.,  0.,  0.,  4., -3.,  6.,
        0.,  0.,  5.,  1.,  2.,  0.,  0.,  5., -2.,  4.,  0.,  0.,  6.,
       -1.,  2.,  0.,  0.,  4.,  3.,  3., 18., 18.,  4., -4.,  4., 32.,
       14.,  5.,  2.,  1.,  6.,  8.,  5., -3.,  0.,  0.,  8.,  6.,  1.,
        0.,  0.,  8.,  6., -2.,  0.,  0.,  8.,  7., -1.,  0.,  0.,  8.,
        5.,  3.,  0.,  0.,  8.,  5., -4.,  0.,  0.,  8.,  6.,  2.,  0.,
        0.,  8.,  6., -3.,  0.,  0.,  8.,  7.,  1.,  0.,  0.,  8.,  7.,
       -2.,  0.,  0.,  8.,  8., -1.,  0.,  0.,  8.,  5.,  4.,  0.,  0.,
        8.,  5., -5.,  0.,  0.,  8.,  6.,  3.,  0.,  0.,  8.,  6., -4.,
        0.,  0.,  8.,  7.,  2.,  0.,  0.,  8.,  7., -3.,  0.,  0.,  8.,
        8.,  1.,  0.,  0.,  8.,  8., -2.,  0.,  0.,  8.,  9., -1.,  0.,
        0.,  8.,  6.,  4.,  0.,  0.,  8.,  6., -5.,  0.,  0.,  8.,  7.,
        3.,  0.,  0.,  8.,  7., -4.,  0.,  0.,  8.,  8.,  2.,  0.,  0.,
        8.,  8., -3.,  0.,  0.,  8.,  9.,  1.,  0.,  0.,  8.,  9

In [20]:
qwrr = gdp.parse_csf_2_descriptor(peel_subshells, csf_data)

In [21]:
qwrr

array([ 2.,  0.,  0.,  4.,  0.,  0.,  6.,  0.,  0.,  2.,  0.,  0.,  4.,
        0.,  0.,  2.,  0.,  0.,  3.,  9.,  9.,  4., 16.,  7.,  1.,  3.,
        4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,
        0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,
        0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,
        4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,
        0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,
        0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,
        4.,  0.,  0.,  4.,  0.,  0.,  4.,  0.,  0.,  4.], dtype=float32)

In [13]:
len(peel_subshells)

95

In [16]:
peel_subshells

['5s',
 '4d-',
 '4d',
 '5p-',
 '5p',
 '6s',
 '4f-',
 '4f',
 '5d-',
 '5d',
 '6p-',
 '6p',
 '7s',
 '5f-',
 '5f',
 '6d-',
 '6d',
 '7p-',
 '7p',
 '8s',
 '5g-',
 '5g',
 '6f-',
 '6f',
 '7d-',
 '7d',
 '8p-',
 '8p',
 '9s',
 '6g-',
 '6g',
 '7f-',
 '7f',
 '8d-',
 '8d',
 '9p-',
 '9p',
 '10s']

In [ ]:
test = gdp.EnergyFile2csv.from_filepath('/home/workstation2/nvmesdd2T/graspcal/Gd/GdI/cv2odd1/test.lev', '/home/workstation2/nvmesdd2T/graspcal/Gd/GdI/cv2odd1')

In [2]:
mixfile = gdp.GraspFileLoad.from_filepath('/home/workstation3/caldata/GdI/vvoddI/oddIvvas1.cm', 'CMIX')

Data file /home/workstation3/caldata/GdI/vvoddI/oddIvvas1.cm loaded.


In [ ]:
mixfile = gdp.GraspFileLoad.from_filepath('/home/workstation3/caldata/GdI/vvoddI/oddIvvas1.cm', 'CMIX')

In [16]:
orbs = gdp.extract_orbs("/home/workstation2/nvmesdd2T/graspcal/Gd/Gdmlci/as3_odd2/mJ-1-90chosenas3_odd2.c")

In [83]:
num_line = csffiledata.CSFs_block_data[0][0][0].rstrip()


In [76]:
J2nu_line = csffiledata.CSFs_block_data[0][0][1].rstrip().ljust(len(csffiledata.CSFs_block_data[0][0][0].rstrip()))

In [78]:
J2cpl_chunk = csffiledata.CSFs_block_data[0][0][2].rstrip()[5:-4].ljust(len(csffiledata.CSFs_block_data[0][0][0].rstrip()))


In [4]:
import msgpack
with open('D:\\PythonProjects\\as3_odd4\\cv4odd1as3_odd4_1\\cv4odd1as3_odd4_1_chosen_indices.msgpack', 'rb') as f:
    data = msgpack.load(f, strict_map_key=False)

print(data)



{0: [2008036, 2413760, 1332878, 3853991, 317079, 1682444, 414786, 1581139, 2872222, 4195057, 4103763, 3078411, 362091, 2228847, 1700246, 3419904, 165180, 1361280, 2293568, 3171035, 1630439, 2715621, 3993712, 3604696, 4272595, 749538, 99798, 1311759, 3429076, 2469463, 1243586, 3724947, 3363089, 935013, 474714, 458229, 3024063, 4114458, 924395, 970798, 982902, 591499, 1145727, 3701442, 2356459, 354734, 1371029, 3682667, 144231, 969184, 236759, 903520, 1189954, 1907451, 326317, 2981263, 3992821, 3054858, 824173, 1386332, 3611060, 1353865, 3293311, 3688588, 3580556, 291769, 3880735, 3026722, 3794152, 3905999, 3464260, 2377256, 2203172, 330911, 1679716, 3639055, 1804242, 3483239, 2322920, 608663, 3871823, 3254106, 2289472, 2153551, 3662061, 2466261, 1219124, 1557430, 1934658, 2984291, 2495950, 2123020, 1456320, 863980, 2194493, 1024547, 1001729, 2673970, 127900, 2295807, 3341681, 403690, 2757262, 2557766, 3109954, 2803417, 4040803, 2824624, 2034249, 1019186, 1408036, 3322564, 1289784, 59490

{0: [2008036,
  2413760,
  1332878,
  3853991,
  317079,
  1682444,
  414786,
  1581139,
  2872222,
  4195057,
  4103763,
  3078411,
  362091,
  2228847,
  1700246,
  3419904,
  165180,
  1361280,
  2293568,
  3171035,
  1630439,
  2715621,
  3993712,
  3604696,
  4272595,
  749538,
  99798,
  1311759,
  3429076,
  2469463,
  1243586,
  3724947,
  3363089,
  935013,
  474714,
  458229,
  3024063,
  4114458,
  924395,
  970798,
  982902,
  591499,
  1145727,
  3701442,
  2356459,
  354734,
  1371029,
  3682667,
  144231,
  969184,
  236759,
  903520,
  1189954,
  1907451,
  326317,
  2981263,
  3992821,
  3054858,
  824173,
  1386332,
  3611060,
  1353865,
  3293311,
  3688588,
  3580556,
  291769,
  3880735,
  3026722,
  3794152,
  3905999,
  3464260,
  2377256,
  2203172,
  330911,
  1679716,
  3639055,
  1804242,
  3483239,
  2322920,
  608663,
  3871823,
  3254106,
  2289472,
  2153551,
  3662061,
  2466261,
  1219124,
  1557430,
  1934658,
  2984291,
  2495950,
  2123020,
  1456320

In [84]:
num_chunk = gdp.chunkit(num_line, 9)


In [99]:
num_chunk[0][:5][3:5]

's '

In [96]:
maxnum = gdp.get_max_num(num_chunk[0][:5][3:5])

In [97]:
maxnum

2

In [80]:
J2nu_chunk = gdp.chunkit(J2nu_line, 9)

In [81]:
J2cpl_chunk = gdp.chunkit(J2cpl_chunk, 9)


In [88]:
J2nu = J2nu_chunk[4].split(';')

In [108]:
csffiledata.CSFs_block_length[0]


np.int64(338516)

In [89]:
csffiledata.CSFs_block_data[0][0]

['  5s ( 2)  4d-( 4)  4d ( 6)  5p-( 2)  5p ( 4)  6s ( 2)  4f-( 3)  4f ( 4)  5d-( 1)\n',
 '                                                            9/2        8      3/2\n',
 '                                                                        7/2      2-\n']

In [101]:
N_ci, cis_ts, head, indexs = gdp.get_total_ci("/home/workstation2/nvmesdd2T/graspcal/Gd/Gdmlci/as3_odd2/mJ-1-90chosenas3_odd2.c")

In [107]:
N_ci

[338516]

In [100]:
[1, max(1, 12 - 2* 2)]


[1, 8]

In [91]:
spsps

['5s',
 '4d-',
 '4d',
 '5p-',
 '5p',
 '6s',
 '4f-',
 '4f',
 '5d-',
 '5d',
 '6p-',
 '6p',
 '7s',
 '5f-',
 '5f',
 '6d-',
 '6d',
 '7p-',
 '7p',
 '8s',
 '5g-',
 '5g',
 '6f-',
 '6f',
 '7d-',
 '7d',
 '8p-',
 '8p',
 '9s']

In [92]:
gdp.lines_to_csf(csffiledata.subshell_info_raw[-1], csffiledata.CSFs_block_data[0][0], 2)

[2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 9,
 9,
 0,
 0,
 9,
 0,
 0,
 9,
 0,
 0,
 9,
 0,
 0,
 9,
 4,
 16,
 7,
 0,
 0,
 7,
 0,
 0,
 7,
 0,
 0,
 7,
 0,
 0,
 7,
 1,
 3,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 2,
 0,

In [5]:
energylist.energy_file2csv()

file /home/workstation2/nvmesdd2T/graspcal/Gd/GdI/cv2odd1/test.lev loaded
data file type: level data
energy levels data has been written into LEVEL/test.lev_level.csv csv file


PosixPath('LEVEL/test.lev_level.csv')

In [9]:
test2 = {"atom": "GdI", 
              # "file_dir": "C:\\Users\\Yenoch\\ProjectFiles\\Python_Project", 
            #   "file_dir": "D:\\test\\Gd_cal_data\\GdIoddIcvIIas3.c", 
            #   "file_dir": "D:\\DataFiles\\GdIoddIcvIIas01\\GdIoddIcvIIas1.c", 
              "file_dir": "/home/workstation2/nvmesdd2T/graspcal/Gd/Gdmlci/as3_odd2/mJ-1-90chosenas3_odd2.c", 
            
              "file_name": "mJ-1-90chosenas3_odd2.c", 
              "level_parameter": "", 
              "this_as": 3, 
              "file_type": "CSF"}

In [3]:
test3 = {"atom": "GdI", 
            #   "file_dir": "C:\\Users\\Yenoch\\ProjectFiles\\Python_Project", 
              # "file_dir": "D:\\test\\Gd_cal_data\\GdIoddIcvIIas2.c", 
            #   "file_dir": "D:\\DataFiles\\GdIoddIcvIIas01\\GdIoddIcvIIas1.c", 
              # "file_dir": "/home/workstation3/caldata/GdI/cvodd1/cv4odd1as2raw.c", 
              "file_dir": "/home/workstation2/nvmesdd2T/graspcal/Gd/GdI/cv2odd1/mJ-1-90chosenas3.c", 
              "file_name": "mJ-1-90chosenas3.c", 
              "level_parameter": "", 
              "this_as": 2, 
              "file_type": "CSF"}

In [ ]:
test3 = {"atom": "GdI", 
            #   "file_dir": "C:\\Users\\Yenoch\\ProjectFiles\\Python_Project", 
            #   "file_dir": "D:\\test\\Gd_cal_data\\GdIoddIcvIIas3.c", 
            #   "file_dir": "D:\\DataFiles\\GdIoddIcvIIas01\\GdIoddIcvIIas1.c", 
              "file_dir": "/home/workstation3/caldata/GdI/cvodd1/cv4odd1as3raw.c", 
              "file_name": "cv4odd1as3raw.c", 
              "level_parameter": "", 
              "this_as": 3, 
              "file_type": "CSF"}

In [10]:
csffiledata = gdp.GraspFileLoad(test2).data_file_process()


Data file /home/workstation2/nvmesdd2T/graspcal/Gd/Gdmlci/as3_odd2/mJ-1-90chosenas3_odd2.c loaded.


In [11]:
csffiledata.CSFs_block_length

array([338516])

In [9]:
import math
np.ceil(csffiledata.CSFs_block_length * 0.9)

array([304665.])

In [5]:
csffiledata_as3 = gdp.GraspFileLoad(test3).data_file_process()

Data file /home/workstation2/nvmesdd2T/graspcal/Gd/GdI/cv2odd1/mJ-1-90chosenas3.c loaded.


In [ ]:
csffiledata_as3.

In [9]:
csffiledata_as3.CSFs_block_length * 0.9


TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
blocks_coupling_as3 = gdp.batch_blocks_csfs_final_coupling_J_collection(csffiledata_as3.CSFs_block_data)


In [ ]:
index_mapping = {value: idx for idx, value in enumerate(blocks_coupling_as3)}
small_list_indices = [index_mapping[value] for value in blocks_coupling if value in index_mapping]

In [ ]:
for item, value in block1.items():
    print(f"{item}: {value['count']}") 
    print(f"{item}: {value['indices']}") 

In [ ]:
gdp.write_sorted_CSFs_to_cfile(csffiledata.subshell_info_raw, csffiledata.CSFs_block_data, "D:\\test\\Gd_cal_data\\testaaset.c")

In [ ]:
test = gdp.batch_blocks_CSFs_final_coupling_J_collection(csffiledata.CSFs_block_data, cmfiledata.mix_coefficient_list, coupling_level=2)

In [ ]:
gdp.CSFs_sort_by_mix_coefficient(csffiledata.CSFs_block_data[0], cmfiledata.mix_coefficient_list[0][0])

In [ ]:
cmfiledata.mix_coefficient_list

In [ ]:
cmfiledata.block_index_list

In [ ]:
gdp.csf_mix_data_abs_above_threshold(cmfiledata)

In [ ]:
coupling_level = 2
CSFs_choosed_coupling_info = [CSF_coupling_info[-coupling_level:] if len(CSF_coupling_info) >= coupling_level else CSF_coupling_info for CSF_coupling_info in CSFs_coupling_info_list]


# result = [sublist[-n:] if len(sublist) >= n else sublist for sublist in block1_csf_3line]

In [ ]:
block1_counts = gdp.CSF_final_coupling_J_collection(csffiledata.CSFs_block_data[0], 2)

In [ ]:
block1_counts

In [ ]:
ci_list = []
for index, element in enumerate(block1_counts):
    print(f"元素 {element} 出现次数: {block1_counts[element]['count']}")
    sum_ci = 0
    for ci in block1_counts[element]['indices']:
        sum_ci += cmfiledata.mix_coefficient_list[0][0][ci]**2
    print(f"元素 {element} 对应的索引之和: {sum_ci}")
    ci_list.append(sum_ci)

In [ ]:
lll = ('2', '3/2', '1-')

In [ ]:
for index, item in enumerate(block1_counts):
    print(f"元素 {item} 出现次数: {block1_counts[item]['count']}, 索引: {block1_counts[item]['indices']}")
    

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

# 查找系统支持的中文字体
font_list = [f.name for f in matplotlib.font_manager.fontManager.ttflist if 'Hei' in f.name or 'Sun' in f.name or 'Kai' in f.name]
print("可用中文字体:", font_list)

# 选择其中一个字体
plt.rcParams['font.sans-serif'] = [font_list[0]] if font_list else ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimSun'] 

plt.rcParams['font.family'] = 'Times New Roman'  # or 'Arial', 'Times New Roman'
# 数据准备
elements = ['3/2', '5/2', '1/2', '1', '7/2', '3', '2', '4', '5', '1-', '9/2', '11/2']
counts = [13114, 6758, 6697, 912, 862, 842, 519, 210, 195, 86, 31, 25]
index_sums = [0.0081, 0.0158, 0.0029, 0.00014, 0.8836, 0.0759, 0.00012, 7.75e-5, 4.24e-5, 0.0132, 8.25e-6, 7.53e-6]

# 创建子图
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# 饼图 - 展示出现次数分布
ax1.pie(counts, 
        labels=elements,
        autopct=lambda p: f'{p:.1f}% ({int(p*sum(counts)/100)})',
        startangle=90,
        wedgeprops={'linewidth': 1, 'edgecolor': 'white'})
# ax1.set_title("元素出现次数分布")

# 条形图 - 展示索引之和对比
ax2.bar(elements, index_sums, color='skyblue')
# ax2.set_title("元素索引之和对比")
ax2.set_yscale('log')  # 使用对数坐标轴
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()
plt.savefig('E:\\LaTeX\\team_report\\20250422\\figure1.png')

In [ ]:
import matplotlib.pyplot as plt

# 取前1000个最大的值
top_values = ci_list

# 创建饼图
plt.figure(figsize=(10, 8))
plt.pie(top_values, 
        labels=[f"值{i+1}" for i in range(len(top_values))],  # 自动生成标签
        autopct='%1.1f%%',  # 显示百分比
        startangle=90,  # 起始角度
        wedgeprops={'linewidth': 1, 'edgecolor': 'white'})  # 设置边框

plt.title("Top 1000 值分布饼图")
plt.axis('equal')  # 保证饼图是圆形
plt.show()

In [ ]:
for index, element in enumerate(block1_counts):
    if index == 284:
        print(f"元素 {element} 出现次数: {block1_counts[element]['count']}, 索引: {len(block1_counts[element]['indices'])}")
    

In [ ]:
len(ci_list)

In [ ]:
sorted_ci = sorted(ci_list, reverse=True)

In [ ]:
sum(sorted_ci[:1000])

In [ ]:
sorted_ci[:1000]

In [ ]:
ci_list.index(max(ci_list))

In [ ]:
ci_list.sort(reverse=True)

In [ ]:
ci_list

In [ ]:
cmfiledata.mix_coefficient_list[0][0]

In [ ]:
llll = gdp.level_mix_data_abs_above_threshold(cmfiledata.mix_coefficient_list[0][0])

In [ ]:
llll

In [ ]:
gdp.csf_mix_data_abs_above_threshold(cmfiledata, 0.01)

In [ ]:
block1_ci = np.abs(cmfiledata.mix_coefficient_list[0][0])

In [ ]:
np.mean(np.abs(cmfiledata.mix_coefficient_list[0][0]))

In [ ]:
np.sum(block1_ci > 1e-05)

In [ ]:
epsilon = 1e-18
block1_ci_log = np.log(block1_ci + epsilon)

In [ ]:
log_indices = np.floor(np.log10(block1_ci + 1e-30))

In [ ]:
unique_indices, counts = np.unique(log_indices, return_counts=True)
# 计算占比
total = len(block1_ci)
percentages = (counts / total) * 100

# 按指数排序（从小到大）
sorted_indices = np.argsort(unique_indices)
unique_indices = unique_indices[sorted_indices]
percentages = percentages[sorted_indices]

In [ ]:
import matplotlib.pyplot as plt

# 生成标签（例如 "1e-18"）
labels = [f"1e{int(idx)}" for idx in unique_indices]

# 绘制柱状图
plt.figure(figsize=(10, 6))
plt.bar(labels, percentages, color='skyblue')
# plt.xlabel("数量级")
plt.ylabel("persontage (%)")
plt.title("block1_ci")
plt.xticks(rotation=45)
plt.show()

In [ ]:
for v in block1_counts.values():
    print(v)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 准备数据
elements = list(block1_counts.keys())
counts = [v['count'] for v in block1_counts.values()]  # 出现次数
indices = [v['indices'] for v in block1_counts.values()]  # 对应的索引数组

# 计算每个元素对应的block1_ci_log平均值
avg_values = [np.mean(block1_ci_log[idx]) for idx in indices]

# 创建图形
fig, ax1 = plt.subplots(figsize=(10, 6))

# 第一个y轴（频率占比）
ax1.bar(elements, np.array(counts)/sum(counts), alpha=0.6, color='b', label='Frequency')
ax1.set_ylabel('Frequency Proportion', color='b')
ax1.tick_params(axis='y', labelcolor='b')

# 第二个y轴（对应值）
ax2 = ax1.twinx()
ax2.plot(elements, avg_values, 'r-', marker='o', label='Avg CI Log Value')
ax2.set_ylabel('Average CI Log Value', color='r')
ax2.tick_params(axis='y', labelcolor='r')

plt.title('Element Frequency and Corresponding CI Values')
fig.legend(loc='upper right')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
avg_ci

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 准备数据
categories = list(block1_counts.keys())
counts = [v['count'] for v in block1_counts.values()]
avg_ci = [np.mean(block1_ci_log[v['indices']]) for v in block1_counts.values()]

# 创建画布和双坐标轴
fig, ax1 = plt.subplots(figsize=(12, 6))

# 左侧y轴 - 计数柱状图
color = 'tab:blue'
ax1.set_xlabel('Categories')
ax1.set_ylabel('Count', color=color)
bars = ax1.bar(categories, counts, color=color, alpha=0.6)
ax1.tick_params(axis='y', labelcolor=color)

# 右侧y轴 - 平均CI值点图
ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Average CI log value', color=color)
points = ax2.scatter(categories, avg_ci, color=color, s=50)
ax2.tick_params(axis='y', labelcolor=color)

# 优化显示
plt.title('Element Distribution with Average CI Values')
plt.xticks(rotation=45, ha='right')
fig.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

# 准备数据
categories = list(block1_counts.keys())
sizes = [v['count'] for v in block1_counts.values()]
avg_ci = [np.mean(block1_ci_log[v['indices']]) for v in block1_counts.values()]

# 创建颜色映射 (使用tab20色标，支持20种不同颜色)
colors = cm.tab20(np.linspace(0, 1, len(categories)))

groups = ['high' if c > 1000 else 'medium' if c > 100 else 'low' for c in sizes]
group_colors = {'high': 'red', 'medium': 'orange', 'low': 'green'}

plt.figure(figsize=(14, 8))
scatter = plt.scatter(range(len(categories)), 
                     avg_ci, 
                     s=[s/10 for s in sizes],
                     c=[group_colors[g] for g in groups],  # 按分组着色
                     alpha=0.7)
for i, cat in enumerate(categories[:3]):  # 只显示前5个避免拥挤
    plt.annotate(cat, (i, avg_ci[i]), textcoords="offset points", xytext=(0,10), ha='center')
# 创建分组图例
import matplotlib.patches as mpatches
legend_elements = [mpatches.Patch(color='red', label='High count (>1000)'),
                  mpatches.Patch(color='orange', label='Medium count (100-1000)'),
                  mpatches.Patch(color='green', label='Low count (<100)')]
plt.legend(handles=legend_elements)

plt.xlabel('Category Index')
plt.ylabel('Average CI log value')
plt.title('Bubble Chart: Size=Count, Y=Avg CI, Color=Count Group')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm

# 准备数据
categories = list(block1_counts.keys())
sizes = [v['count'] for v in block1_counts.values()]
avg_ci = [np.mean(block1_ci_log[v['indices']]) for v in block1_counts.values()]

# 创建颜色映射 (使用tab20色标，支持20种不同颜色)
colors = cm.tab20(np.linspace(0, 1, len(categories)))

groups = ['high' if c > 1000 else 'medium' if c > 100 else 'low' for c in sizes]
group_colors = {'high': 'red', 'medium': 'orange', 'low': 'green'}

plt.figure(figsize=(14, 8))
scatter = plt.scatter(range(len(categories)), 
                     avg_ci, 
                     s=[s/10 for s in sizes],
                     c=sizes,  # 使用count值作为颜色基准
                     cmap='viridis',  # 使用渐变色标
                     alpha=0.7)
for i, cat in enumerate(categories[:3]):  # 只显示前5个避免拥挤
    plt.annotate(cat, (i, avg_ci[i]), textcoords="offset points", xytext=(0,10), ha='center')
# 添加颜色条
cbar = plt.colorbar()
cbar.set_label('Count Value')

plt.xlabel('Category Index')
plt.ylabel('Average CI log value')
plt.title('Bubble Chart: Size=Count, Y=Avg CI, Color=Count Value')
plt.grid(True)
plt.show()

np.std(block1_ci) 是 NumPy 库中的函数调用，np 是 NumPy 库的常用别名，std 是 standard deviation（标准差）的缩写，该函数的作用是计算 block1_ci 数组的标准差。

标准差的定义
标准差是用来衡量一组数据的离散程度，即数据相对于其均值的分散程度。标准差越大，说明数据越分散；标准差越小，说明数据越集中在均值附近。其数学公式如下： [ \sigma = \sqrt{\frac{1}{N} \sum_{i=1}^{N} (x_i - \mu)^2} ] 其中，(\sigma) 表示标准差，(N) 是数据点的数量，(x_i) 是第 (i) 个数据点，(\mu) 是数据的均值。

In [ ]:
np.std(block1_ci)

In [ ]:
block1_ci_log.max()

In [ ]:
block1_ci_log.min()

In [ ]:
block1_ci_log.mean()

In [ ]:
cmfiledata.ncfblk_list[0]

In [ ]:
theta = np.linspace(0.0, 2 * np.pi, cmfiledata.ncfblk_list[0], endpoint=False)


In [ ]:
theta

In [ ]:
for element, info in block1_counts.items():
    # if element not in color_map:
        # 为新的 element 随机生成一个颜色
        # color_map[element] = plt.cm.viridis(random())
    print(f"元素 {element} 出现次数: {info['count']}, 索引: {info['indices']}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from random import random
# Fixing random state for reproducibility
# np.random.seed(19680801)

# Compute pie slices
N = cmfiledata.ncfblk_list[0]

# 不添加偏移，直接从 0 开始生成角度
theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)
color_map = {}
for element, info in block1_counts.items():
    if element not in color_map:
        # 为新的 element 随机生成一个颜色
        color_map[element] = plt.cm.viridis(random())
    # print(f"元素 {element} 出现次数: {info['count']}, 索引: {info['indices']}")
    for i in info['indices']:
        print(i)
        radii = block1_ci[i]
        width = np.pi / N
        colors = color_map[element]

        ax = plt.subplot(projection='polar')
    ax.bar(theta, radii, width=width, bottom=0.0, color=colors, alpha=0.5)
        


# 确保 0° 位置在正东方向
ax.set_theta_zero_location('N')
# 确保角度逆时针旋转
ax.set_theta_direction(1)


# 自定义半径刻度位置
# ax.set_yticks([0, 2, 4, 6, 8, 10])
# 自定义半径刻度标签
# ax.set_yticklabels(['Min', '2', '4', '6', '8', 'Max'])

# 隐藏角度刻度标签
ax.set_xticklabels([])

plt.show()

In [ ]:
ind = gdp.level_mix_data_abs_above_threshold(cmfiledata['mix_coefficient_list'][1][0], threshold=0.00001)

In [ ]:
len(ind)/len(cmfiledata['mix_coefficient_list'][1][0])

In [ ]:
ind

In [ ]:
gdp.level_mix_above_threshold_coupling_info(ind, csffiledata['CSFs_block_data'][1])[2]

In [ ]:
cccc['CSFs_block_data'][0]

In [ ]:
sc = gdp.CSFs_block_get_CSF(cccc['CSFs_block_data'][0], (np.int64(5),))

In [ ]:
# 示例数据（假设三个 CSF）
csf_data = [
    {"identifier": "2+", "coeff": 0.8},
    {"identifier": "2+", "coeff": 0.6},
    {"identifier": "1/2 2+", "coeff": 0.45}
]

# 统计出现次数和系数绝对值之和
from collections import defaultdict
stats = defaultdict(lambda: {"count": 0, "coeff_sum": 0.0})
for csf in csf_data:
    id = csf["identifier"]
    stats[id]["count"] += 1
    stats[id]["coeff_sum"] += abs(csf["coeff"])  # 取绝对值求和

# 提取标签和数据
identifiers = list(stats.keys())
counts = [stats[id]["count"] for id in identifiers]
coeff_sums = [stats[id]["coeff_sum"] for id in identifiers]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Fixing random state for reproducibility
np.random.seed(19680801)

# Compute pie slices
N = 20000
# 不添加偏移，直接从 0 开始生成角度
theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)
radii = 10 * np.random.rand(N)
width = np.pi / 4 * np.random.rand(N)
colors = plt.cm.viridis(radii / 10.)

ax = plt.subplot(projection='polar')
# 确保 0° 位置在正东方向
ax.set_theta_zero_location('N')
# 确保角度逆时针旋转
ax.set_theta_direction(1)

ax.bar(theta, radii, width=width, bottom=0.0, color=colors, alpha=0.5)

# 自定义半径刻度位置
ax.set_yticks([0, 2, 4, 6, 8, 10])
# 自定义半径刻度标签
ax.set_yticklabels(['Min', '2', '4', '6', '8', 'Max'])

# 隐藏角度刻度标签
ax.set_xticklabels([])

plt.show()

以上图为例，读取rmix文件，对应CSFs的ci系数，找出CSFs的第三行作为分类，将ci按从小到大排列，进而画出饼图并拼接在一起。

In [ ]:
gdp.CSF_item_2_dict(sc)

In [ ]:
for i in range(0,len(cccc['CSFs_block_data'][0]),3):
    print(cccc['CSFs_block_data'][0][i])

In [ ]:
cccc['CSFs_block_data'][0]

In [ ]:
mix_data_dim

In [ ]:
choosen_mix_data[0]

In [ ]:
len(cmfiledata['mix_coefficient_list'][2])

In [ ]:
jjjj = gdp.mix_data_abs_above_threshold(cmfiledata['mix_coefficient_list'][0][0], 0.01)

In [ ]:
indices = [idx[0] for idx in jjjj.keys()]

In [ ]:
for index in jjjj.items():
    print(f"Index: {index}, Value")

In [ ]:
test1 = {"atom": "GdI", 
              "file_dir": "D:\\test\\Gd_cal_data\\cvas2069as2", 
              "file_name": "Gdcvas2069as2level", 
              "level_parameter": "", 
              "this_as": 2, 
              "cut_off_subshell": "5p", 
              "file_type": "level"}

In [ ]:
from typing import Dict, Tuple, List



In [ ]:
test3 = {"atom": "GdIoddIcvIItest", 
              "file_dir": "D:\\test\\Gd_cal_data\\cvas2069as2", 
              "file_name": "", 
              "level_parameter": "as2069as", 
              "this_as": 2, 
              "cut_off_subshell": "5p", 
              "file_type": "TRANSITION"}

In [ ]:
leveldata = gdp.LevelsASFComposition(leveldata, test3)

In [ ]:
leveldata = gdp.ci_energy_data_collection(None, test1)

In [ ]:
test2 = {"atom": "GdIoddIcvIItest", 
              "file_dir": "D:\\test\\Gd_cal_data\\cvas2069as2", 
              "file_name": "", 
              "level_parameter": "as2069", 
              "this_as": 2, 
              "cut_off_subshell": "5p", 
              "file_type": "TRANSITION"}

In [ ]:
transition_data_collection = gdp.TransitionDataCollection(test2)

data_pd = transition_data_collection.transition_data2dataframe()